In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf 
import keras
from tensorflow.keras.utils import Sequence, to_categorical, plot_model
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, UpSampling2D, concatenate, Input, Dense
from tensorflow.keras.models import Model,Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# Importing all the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf 
import keras
from tensorflow.keras.utils import Sequence, to_categorical, plot_model
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, UpSampling2D, concatenate, Input, Dense,BatchNormalization,Activation,MaxPool2D
from tensorflow.keras import layers
from albumentations import HorizontalFlip, RandomRotate90, RandomBrightnessContrast
from tensorflow.keras.models import Model,Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, Callback


# Import data 

In [ ]:
path = '/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/'
img = cv2.imread(path + 'original_images/001.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
mask = cv2.imread(path + 'label_images_semantic/001.png', cv2.IMREAD_GRAYSCALE)
#mask = mask.cvtColor(img, cv2.COLOR_BGR2RGB)
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].imshow(img)
axs[1].imshow(mask)

In [ ]:
path = '/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/'
img = cv2.imread(path + 'original_images/001.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
flippedimage= tf.keras.preprocessing.image.random_rotation( img, 90, row_axis=1, col_axis=2, channel_axis=0, fill_mode='nearest',cval=0.0, interpolation_order=1)
flippedmask=cv2.flip(mask,1)
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].imshow(flippedimage)
axs[1].imshow(flippedmask)

In [ ]:
n = 128
# Prepare the Images
X = []
for filename in tqdm(sorted(os.listdir(path + 'original_images/'))):
    a = cv2.imread(path + 'original_images/' + filename)
    a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    a = cv2.resize(a, (n, n))
    a = a / 255
    X.append(a)
    
X = np.array(X)

# Prepare the Labels
Y = []
for filename in tqdm(sorted(os.listdir(path + 'label_images_semantic/'))):
    a = cv2.imread(path + 'label_images_semantic/' + filename, cv2.IMREAD_GRAYSCALE)
    a = cv2.resize(a, (n, n))
    Y.append(a)
    
Y = np.array(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.20, random_state=42)

In [ ]:
X_train = X_train.reshape(-1,16384,3)
X_test = X_test.reshape(-1,16384,3)


In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
z_train = to_categorical(y_train).reshape(-1,16384,23)
z_test = to_categorical(y_test).reshape(-1,16384,23)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
input_img = keras.Input(shape=(16384,3,))
x = Dense(500, activation='relu')(input_img)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(23, activation='sigmoid')(x)

model = Model(input_img, x)
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy',metrics=[tf.keras.metrics.MeanIoU(num_classes=23),'accuracy'])

history=model.fit(X_train, z_train,
                epochs=50,
                batch_size=16,
                shuffle=True,
                validation_data=(X_test, z_test))

In [ ]:
pred = np.argmax(model.predict(X_test),axis=2).reshape(-1,128,128)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 10))
t = X_test[1].reshape(128,128,3)
t1 = pred[1].reshape(128,128)
t2 = y_test[1]
axs[0].imshow(t)
axs[1].imshow(t1)
axs[2].imshow(t2)

In [ ]:
y_test[1].shape

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'Test'], loc='upper left')
plt.show()